Rarity Analysis will look at items listed in the catalog and measures how difficult it would be to acquire. 
"Items" include: accessories, art, bags, dress-up, floors, headwear, bottoms, housewares, posters.
This notebook does the following:
1. Defines items (by exploring the available catalogs. e.g. a "Reaction" is not an item, a "bag" is an item. Only the latter will be evaluated for rarity.)
3. Defines criteria to be measured against to determine rarity (by exploring the columns in each catalog. Any dimension that indicates availability/restrictions/etc. could factor into the rarity calculation.)
4. Determine the rarity calculation (based on the values within relevant columns, what items are rare and what items are common?) 

In [1]:
import os
import sys
import pandas as pd
os.chdir('..') #change the current working directory
sys.path.append(os.getcwd()) #add diretory to the path

In [2]:
from load_python import load_data
data = load_data(data_folder='data')

Loaded accessories with 222 rows and 22 columns.
Loaded achievements with 84 rows and 21 columns.
Loaded art with 70 rows and 26 columns.
Loaded bags with 96 rows and 20 columns.
Loaded bottoms with 726 rows and 20 columns.
Loaded construction with 236 rows and 7 columns.
Loaded dress-up with 913 rows and 22 columns.
Loaded fencing with 19 rows and 11 columns.
Loaded fish with 80 rows and 41 columns.
Loaded floors with 176 rows and 19 columns.
Loaded fossils with 73 rows and 14 columns.
Loaded headwear with 698 rows and 22 columns.
Loaded housewares with 3275 rows and 32 columns.
Loaded insects with 80 rows and 38 columns.
Loaded miscellaneous with 1307 rows and 31 columns.
Loaded music with 98 rows and 13 columns.
Loaded other with 353 rows and 15 columns.
Loaded photos with 3128 rows and 20 columns.
Loaded posters with 452 rows and 13 columns.
Loaded reactions with 44 rows and 5 columns.
Loaded recipes with 595 rows and 24 columns.
Loaded rugs with 132 rows and 19 columns.
Loaded sho

Look into each item DataFrame to understand what columns we have.

In [3]:
#Accessories
accessories_df = data['accessories'] #access the Accessories data from the dictionary
print(accessories_df.columns) #retrieve the column names


Index(['Name', 'Variation', 'DIY', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size',
       'Miles Price', 'Source', 'Source Notes', 'Seasonal Availability',
       'Mannequin Piece', 'Version', 'Style', 'Label Themes', 'Type',
       'Villager Equippable', 'Catalog', 'Filename', 'Internal ID',
       'Unique Entry ID'],
      dtype='object')


In [4]:
#Art
art_df = data['art'] #access the item's data from the dictionary
print(art_df.columns) #retrieve the column names


Index(['Name', 'Genuine', 'Category', 'Buy', 'Sell', 'Color 1', 'Color 2',
       'Size', 'Real Artwork Title', 'Artist', 'Museum Description', 'Source',
       'Source Notes', 'Version', 'HHA Concept 1', 'HHA Concept 2',
       'HHA Series', 'HHA Set', 'Interact', 'Tag', 'Speaker Type',
       'Lighting Type', 'Catalog', 'Filename', 'Internal ID',
       'Unique Entry ID'],
      dtype='object')


In [5]:
#Bags
bags_df = data['bags'] #access the item's data from the dictionary
print(bags_df.columns) #retrieve the column names


Index(['Name', 'Variation', 'DIY', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size',
       'Miles Price', 'Source', 'Source Notes', 'Seasonal Availability',
       'Version', 'Style', 'Label Themes', 'Villager Equippable', 'Catalog',
       'Filename', 'Internal ID', 'Unique Entry ID'],
      dtype='object')


In [6]:
#Bottoms
bottoms_df = data['bottoms'] #access the item's data from the dictionary
print(bottoms_df.columns) #retrieve the column names


Index(['Name', 'Variation', 'DIY', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size',
       'Source', 'Source Notes', 'Seasonal Availability', 'Mannequin Piece',
       'Version', 'Style', 'Label Themes', 'Villager Equippable', 'Catalog',
       'Filename', 'Internal ID', 'Unique Entry ID'],
      dtype='object')


In [7]:
#dress-up
dressup_df = data['dress-up'] #access the item's data from the dictionary
print(dressup_df.columns) #retrieve the column names


Index(['Name', 'Variation', 'DIY', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size',
       'Source', 'Source Notes', 'Seasonal Availability', 'Mannequin Piece',
       'Version', 'Style', 'Label Themes', 'Villager Equippable', 'Catalog',
       'Primary Shape', 'Secondary Shape', 'Filename', 'Internal ID',
       'Unique Entry ID'],
      dtype='object')


In [8]:
#floors
floors_df = data['floors'] #access the item's data from the dictionary
print(floors_df.columns) #retrieve the column names


Index(['Name', 'VFX', 'DIY', 'Buy', 'Sell', 'Color 1', 'Color 2',
       'Miles Price', 'Source', 'Source Notes', 'Version', 'HHA Concept 1',
       'HHA Concept 2', 'HHA Series', 'Tag', 'Catalog', 'Filename',
       'Internal ID', 'Unique Entry ID'],
      dtype='object')


In [9]:
#headwear
headwear_df = data['headwear'] #access the item's data from the dictionary
print(headwear_df.columns) #retrieve the column names


Index(['Name', 'Variation', 'DIY', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size',
       'Miles Price', 'Source', 'Source Notes', 'Seasonal Availability',
       'Mannequin Piece', 'Version', 'Style', 'Label Themes', 'Type',
       'Villager Equippable', 'Catalog', 'Filename', 'Internal ID',
       'Unique Entry ID'],
      dtype='object')


In [10]:
#housewares
housewares_df = data['housewares'] #access the item's data from the dictionary
print(housewares_df.columns) #retrieve the column names


Index(['Name', 'Variation', 'Body Title', 'Pattern', 'Pattern Title', 'DIY',
       'Body Customize', 'Pattern Customize', 'Kit Cost', 'Buy', 'Sell',
       'Color 1', 'Color 2', 'Size', 'Miles Price', 'Source', 'Source Notes',
       'Version', 'HHA Concept 1', 'HHA Concept 2', 'HHA Series', 'HHA Set',
       'Interact', 'Tag', 'Outdoor', 'Speaker Type', 'Lighting Type',
       'Catalog', 'Filename', 'Variant ID', 'Internal ID', 'Unique Entry ID'],
      dtype='object')


In [11]:
#miscellaneous
misc_df = data['miscellaneous'] #access the item's data from the dictionary
print(misc_df.columns) #retrieve the column names


Index(['Name', 'Variation', 'Body Title', 'Pattern', 'Pattern Title', 'DIY',
       'Body Customize', 'Pattern Customize', 'Kit Cost', 'Buy', 'Sell',
       'Color 1', 'Color 2', 'Size', 'Source', 'Source Notes', 'Version',
       'HHA Concept 1', 'HHA Concept 2', 'HHA Series', 'HHA Set', 'Interact',
       'Tag', 'Outdoor', 'Speaker Type', 'Lighting Type', 'Catalog',
       'Filename', 'Variant ID', 'Internal ID', 'Unique Entry ID'],
      dtype='object')


In [12]:
#music
music_df = data['music'] #access the item's data from the dictionary
print(music_df.columns) #retrieve the column names


Index(['Name', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size', 'Source',
       'Source Notes', 'Version', 'Catalog', 'Filename', 'Internal ID',
       'Unique Entry ID'],
      dtype='object')


In [13]:
#other
other_df = data['other'] #access the item's data from the dictionary
print(other_df.columns) #retrieve the column names


Index(['Name', 'DIY', 'Stack Size', 'Buy', 'Sell', 'Miles Price', 'Source',
       'Source Notes', 'Tag', 'Color 1', 'Color 2', 'Version', 'Filename',
       'Internal ID', 'Unique Entry ID'],
      dtype='object')


In [14]:
#photos
photos_df = data['photos'] #access the item's data from the dictionary
print(photos_df.columns) #retrieve the column names


Index(['Name', 'Variation', 'Body Title', 'Pattern', 'Pattern Title', 'DIY',
       'Customize', 'Kit Cost', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size',
       'Source', 'Version', 'Catalog', 'Filename', 'Variant ID', 'Internal ID',
       'Unique Entry ID'],
      dtype='object')


In [15]:
#posters
posters_df = data['posters'] #access the item's data from the dictionary
print(posters_df.columns) #retrieve the column names


Index(['Name', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size', 'Source',
       'Source Notes', 'Version', 'Catalog', 'Filename', 'Internal ID',
       'Unique Entry ID'],
      dtype='object')


In [16]:
#recipes
recipes_df = data['recipes'] #access the item's data from the dictionary
print(recipes_df.columns) #retrieve the column names


Index(['Name', '#1', 'Material 1', '#2', 'Material 2', '#3', 'Material 3',
       '#4', 'Material 4', '#5', 'Material 5', '#6', 'Material 6', 'Buy',
       'Sell', 'Miles Price', 'Source', 'Source Notes', 'Recipes to Unlock',
       'Version', 'Category', 'Serial ID', 'Internal ID', 'Unique Entry ID'],
      dtype='object')


In [17]:
#rugs
rugs_df = data['rugs'] #access the item's data from the dictionary
print(rugs_df.columns) #retrieve the column names


Index(['Name', 'DIY', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size',
       'Miles Price', 'Source', 'Source Notes', 'Version', 'HHA Concept 1',
       'HHA Concept 2', 'HHA Series', 'Tag', 'Catalog', 'Filename',
       'Internal ID', 'Unique Entry ID'],
      dtype='object')


In [18]:
#shoes
shoes_df = data['shoes'] #access the item's data from the dictionary
print(shoes_df.columns) #retrieve the column names


Index(['Name', 'Variation', 'DIY', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size',
       'Miles Price', 'Source', 'Source Notes', 'Seasonal Availability',
       'Mannequin Piece', 'Version', 'Style', 'Label Themes',
       'Villager Equippable', 'Catalog', 'Filename', 'Internal ID',
       'Unique Entry ID'],
      dtype='object')


In [19]:
#socks
socks_df = data['socks'] #access the item's data from the dictionary
print(socks_df.columns) #retrieve the column names


Index(['Name', 'Variation', 'DIY', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size',
       'Miles Price', 'Source', 'Source Notes', 'Seasonal Availability',
       'Mannequin Piece', 'Version', 'Style', 'Label Themes',
       'Villager Equippable', 'Catalog', 'Filename', 'Internal ID',
       'Unique Entry ID'],
      dtype='object')


In [20]:
#tools
tools_df = data['tools'] #access the item's data from the dictionary
print(tools_df.columns) #retrieve the column names


Index(['Name', 'Variation', 'Body Title', 'DIY', 'Customize', 'Kit Cost',
       'Uses', 'Stack Size', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size',
       'Set', 'Miles Price', 'Source', 'Source Notes', 'Version', 'Filename',
       'Variant ID', 'Internal ID', 'Unique Entry ID'],
      dtype='object')


In [21]:
#tops
tops_df = data['tops'] #access the item's data from the dictionary
print(tops_df.columns) #retrieve the column names


Index(['Name', 'Variation', 'DIY', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size',
       'Miles Price', 'Source', 'Source Notes', 'Seasonal Availability',
       'Mannequin Piece', 'Version', 'Style', 'Label Themes',
       'Villager Equippable', 'Catalog', 'Filename', 'Internal ID',
       'Unique Entry ID'],
      dtype='object')


In [22]:
#umbrellas
umbrellas_df = data['umbrellas'] #access the item's data from the dictionary
print(umbrellas_df.columns) #retrieve the column names


Index(['Name', 'DIY', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size',
       'Miles Price', 'Source', 'Source Notes', 'Version',
       'Villager Equippable', 'Catalog', 'Filename', 'Internal ID',
       'Unique Entry ID'],
      dtype='object')


In [23]:
#villagers
villagers_df = data['villagers'] #access the item's data from the dictionary
print(villagers_df.columns) #retrieve the column names


Index(['Name', 'Species', 'Gender', 'Personality', 'Hobby', 'Birthday',
       'Catchphrase', 'Favorite Song', 'Style 1', 'Style 2', 'Color 1',
       'Color 2', 'Wallpaper', 'Flooring', 'Furniture List', 'Filename',
       'Unique Entry ID'],
      dtype='object')


In [24]:
#Wall-mounted
wall_mounted_df = data['wall-mounted'] #access the item's data from the dictionary
print(wall_mounted_df.columns) #retrieve the column names


Index(['Name', 'Variation', 'Body Title', 'Pattern', 'Pattern Title', 'DIY',
       'Body Customize', 'Pattern Customize', 'Kit Cost', 'Buy', 'Sell',
       'Color 1', 'Color 2', 'Size', 'Source', 'Source Notes', 'Version',
       'HHA Concept 1', 'HHA Concept 2', 'HHA Series', 'HHA Set', 'Interact',
       'Tag', 'Outdoor', 'Lighting Type', 'Door Deco', 'Catalog', 'Filename',
       'Variant ID', 'Internal ID', 'Unique Entry ID'],
      dtype='object')


In [25]:
#Wallpaper
wallpaper_df = data['wallpaper'] #access the item's data from the dictionary
print(wallpaper_df.columns) #retrieve the column names


Index(['Name', 'VFX', 'VFX Type', 'DIY', 'Buy', 'Sell', 'Color 1', 'Color 2',
       'Miles Price', 'Source', 'Source Notes', 'Catalog', 'Window Type',
       'Window Color', 'Pane Type', 'Curtain Type', 'Curtain Color',
       'Ceiling Type', 'HHA Concept 1', 'HHA Concept 2', 'HHA Series', 'Tag',
       'Version', 'Filename', 'Internal ID', 'Unique Entry ID'],
      dtype='object')


What columns are relevant to the Rarity Analysis?
1. Seasonal Availability.
2. Source Notes. (Where the item is available, restrictions/source exclusivity.)
3. Body Customize, Pattern Customize (Ability to customize.)
5. Catalog. (whether for sale or not)

In [26]:
#Get all dataframes containing x column
#def get_dfs_with_column(data, column_name): #create the function that'll return relevant dataframes
#    items = []
#    for name, df in data.items():
#        if column_name in df.columns:
#            items.append(name)
#    return items

In [27]:
#Get all dataframes containing Seasonal Availability column
def get_dfs_with_column(data, column_name): #create the function that'll return relevant dataframes
    items = []
    for name, df in data.items():
        if column_name in df.columns:
            items.append(name)
    return items
#call the function
results = get_dfs_with_column(data, 'Seasonal Availability')
print('Dataframes with "Seasonal Availability" Column: ', results)

Dataframes with "Seasonal Availability" Column:  ['accessories', 'bags', 'bottoms', 'dress-up', 'headwear', 'shoes', 'socks', 'tops']


In [28]:
#Get all dataframes containing Source Notes column
def get_dfs_with_column(data, column_name): #create the function that'll return relevant dataframes
    items = []
    for name, df in data.items():
        if column_name in df.columns:
            items.append(name)
    return items
#call the function
results = get_dfs_with_column(data, 'Source Notes')
print('Dataframes with "Source Notes" Column: ', results)

Dataframes with "Source Notes" Column:  ['accessories', 'art', 'bags', 'bottoms', 'dress-up', 'fencing', 'floors', 'headwear', 'housewares', 'miscellaneous', 'music', 'other', 'posters', 'reactions', 'recipes', 'rugs', 'shoes', 'socks', 'tools', 'tops', 'umbrellas', 'wall-mounted', 'wallpaper']


In [29]:
#Get all dataframes containing Body Customize column
def get_dfs_with_column(data, column_name): #create the function that'll return relevant dataframes
    items = []
    for name, df in data.items():
        if column_name in df.columns:
            items.append(name)
    return items
#call the function
results = get_dfs_with_column(data, 'Body Customize')
print('Dataframes with "Body Customize": ', results)

Dataframes with "Body Customize":  ['housewares', 'miscellaneous', 'wall-mounted']


In [30]:
#Get all dataframes containing Pattern Customize column
def get_dfs_with_column(data, column_name): #create the function that'll return relevant dataframes
    items = []
    for name, df in data.items():
        if column_name in df.columns:
            items.append(name)
    return items
#call the function
results = get_dfs_with_column(data, 'Pattern Customize')
print('Dataframes with "Pattern Customize" column: ', results)

Dataframes with "Pattern Customize" column:  ['housewares', 'miscellaneous', 'wall-mounted']


In [31]:
#Get all dataframes containing Catalog column
def get_dfs_with_column(data, column_name): #create the function that'll return relevant dataframes
    items = []
    for name, df in data.items():
        if column_name in df.columns:
            items.append(name)
    return items
#call the function
results = get_dfs_with_column(data, 'Catalog')
print('Dataframes with "Catalog" column: ', results)

Dataframes with "Catalog" column:  ['accessories', 'art', 'bags', 'bottoms', 'dress-up', 'floors', 'fossils', 'headwear', 'housewares', 'miscellaneous', 'music', 'photos', 'posters', 'rugs', 'shoes', 'socks', 'tops', 'umbrellas', 'wall-mounted', 'wallpaper']
